In [ ]:
!pip install -U "torch>=2.4.0" "torchvision>=0.19.0" facenet-pytorch "numpy>=2.0,<2.3" "opencv-python-headless==4.12.0.88"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of facenet-pytorch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.

In [ ]:
import os, pickle
import torch
import numpy as np
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.metrics.pairwise import cosine_similarity
import cv2

In [ ]:
RAW_DIR = "/content/drive/MyDrive/My_Machine_Learning/face_recognition-pending/dataset/faces01"
PROC_DIR = "/content/drive/MyDrive/My_Machine_Learning/face_recognition-pending/dataset/processed_face_images"
EMB_FILE = "/content/drive/MyDrive/My_Machine_Learning/face_recognition-pending/dataset/face_embeddings.pkl"
ALLOWED = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("✅ PyTorch device:", device)


✅ PyTorch device: cuda:0


In [ ]:
mtcnn  = MTCNN(image_size=160, margin=20, device=device, post_process=True)
resnet = InceptionResnetV1(pretrained="vggface2").eval().to(device)

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [ ]:
print("torch:", torch.__version__)
print("torchvision:", __import__('torchvision').__version__)
print("numpy:", np.__version__)
print("opencv:", cv2.__version__)
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi | head -n 20

torch: 2.9.0+cu128
torchvision: 0.24.0+cu128
numpy: 2.2.6
opencv: 4.12.0
CUDA available: True
Wed Oct 29 07:26:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P0             28W /   70W |     224MiB /  15360MiB |      0%      Default |
|                                         |   

In [ ]:
# run this cell only if you wanna check the folder is empty or not
"""
print("PROC_DIR exists:", os.path.exists(PROC_DIR))
if os.path.exists(PROC_DIR):
    people = os.listdir(PROC_DIR)
    print("Folders inside PROC_DIR:", people)
    for p in people:
        pdir = os.path.join(PROC_DIR, p)
        print(p, "->", len(os.listdir(pdir)) if os.path.isdir(pdir) else "not a folder")

"""

'\nprint("PROC_DIR exists:", os.path.exists(PROC_DIR))\nif os.path.exists(PROC_DIR):\n    people = os.listdir(PROC_DIR)\n    print("Folders inside PROC_DIR:", people)\n    for p in people:\n        pdir = os.path.join(PROC_DIR, p)\n        print(p, "->", len(os.listdir(pdir)) if os.path.isdir(pdir) else "not a folder")\n\n'

In [ ]:
#!rm -rf "/content/drive/MyDrive/My_Machine_Learning/face_recognition-pending/dataset/face_embeddings.pkl"
# remove old en=mbedding if requires
if os.path.exists(EMB_FILE):
    os.remove(EMB_FILE)
    print("removed old embeddings file.")


removed old embeddings file.


In [ ]:
def iter_images(folder):
    """Yield (person_name, person_dir, image_files)"""
    for person in sorted(os.listdir(folder)):
        pdir = os.path.join(folder, person)
        if not os.path.isdir(pdir):
            continue
        files = [f for f in sorted(os.listdir(pdir))
                 if not f.startswith(".") and os.path.splitext(f)[1].lower() in ALLOWED]
        if files:
            yield person, pdir, files


In [ ]:
# Process each person folder
"""
for person in sorted(os.listdir(RAW_DIR)):
    pdir = os.path.join(RAW_DIR, person)
    if not os.path.isdir(pdir):
        continue

    out_dir = os.path.join(PROC_DIR, person)
    os.makedirs(out_dir, exist_ok=True)

    print(f"🧍 Detecting faces for: {person}")
    for img_name in os.listdir(pdir):
        img_path = os.path.join(pdir, img_name)
        out_path = os.path.join(out_dir, img_name)
        try:
            img = Image.open(img_path).convert("RGB")
            face = mtcnn(img, save_path=out_path)
            if face is None:
                print(f"⚠️ No face found in {img_name}")
        except Exception as e:
            print(f"❌ Error on {img_name}: {e}")

  """

In [ ]:
# only run this cell to check how many images are there in each folder /
"""
print("Folders in PROC_DIR:", os.listdir(PROC_DIR))
for person in os.listdir(PROC_DIR):
    pdir = os.path.join(PROC_DIR, person)
    print(person, "→", len(os.listdir(pdir)), "images")
"""

In [ ]:
embeddings, names = [], []
BATCH = 64

with torch.no_grad():
    for person, pdir, files in iter_images(PROC_DIR):
        print(f"Processing: {person} ({len(files)} images)")
        batch = []
        for fn in files:
            try:
                img = Image.open(os.path.join(pdir, fn)).convert("RGB")
                t = torch.from_numpy(np.asarray(img)).float().permute(2, 0, 1) / 255.0
                batch.append(t)
                if len(batch) == BATCH:
                    x = torch.stack(batch).to(device)
                    em = resnet(x).cpu().numpy()
                    em = em / np.linalg.norm(em, axis=1, keepdims=True)
                    embeddings.append(em)
                    names.extend([person] * len(batch))
                    batch = []
            except Exception as e:
                print("Skipped:", fn, "Reason:", e)

        # leftover images
        if batch:
            x = torch.stack(batch).to(device)
            em = resnet(x).cpu().numpy()
            em = em / np.linalg.norm(em, axis=1, keepdims=True)
            embeddings.append(em)
            names.extend([person] * len(batch))

# Combine and save
emb_array = np.vstack(embeddings).astype(np.float32) if embeddings else np.zeros((0,512), np.float32)

with open(EMB_FILE, "wb") as f:
    pickle.dump({
        "embeddings": emb_array,
        "names": names,
        "model": "InceptionResnetV1-vggface2",
        "normalized": True
    }, f)

print(f"Saved embeddings to: {EMB_FILE}")
print(f"Shape: {emb_array.shape}, Total names: {len(names)}")

Processing: Ashutosh (4 images)


/tmp/ipython-input-2582928134.py:11: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:206.)
  t = torch.from_numpy(np.asarray(img)).float().permute(2, 0, 1) / 255.0


Processing: Chachu (4 images)
Processing: Mom (3 images)
Processing: Muskaa (4 images)
Processing: Naitik (5 images)
Processing: Papa (3 images)
Processing: Vishal (5 images)
Saved embeddings to: /content/drive/MyDrive/My_Machine_Learning/face_recognition-pending/dataset/face_embeddings.pkl
Shape: (28, 512), Total names: 28


In [ ]:
with open(EMB_FILE, "rb") as f:
    data = pickle.load(f)

stored_embeddings = np.array(data["embeddings"])
stored_names = np.array(data["names"])

print("Embeddings and names loaded successfully!")
print("Total embeddings:", len(stored_embeddings))
for i in range(min(5, len(stored_names))):
    print(f"{i+1}. {stored_names[i]}  →  norm: {np.linalg.norm(stored_embeddings[i]):.4f}")


✅ Embeddings and names loaded successfully!
Total embeddings: 28
1. Ashutosh  →  norm: 1.0000
2. Ashutosh  →  norm: 1.0000
3. Ashutosh  →  norm: 1.0000
4. Ashutosh  →  norm: 1.0000
5. Chachu  →  norm: 1.0000


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from facenet_pytorch import InceptionResnetV1, MTCNN

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
mtcnn = MTCNN(image_size=160, margin=20, device=device, post_process=True)
resnet = InceptionResnetV1(pretrained="vggface2").eval().to(device)

def get_face_embedding(image_path):
    """Extract a 512-dim embedding from an image."""
    img = Image.open(image_path).convert("RGB")
    face = mtcnn(img)
    if face is None:
        print("No face detected.")
        return None
    with torch.no_grad():
        embeddings = resnet(face.unsqueeze(0).to(device)).cpu().numpy()[0]
        embeddings = embeddings / np.linalg.norm(embeddings)
    return embeddings

In [ ]:
stored_embeddings = embeddings
stored_names = names


# ====== Main Recognition Function ======
def recognize_face(image_path, threshold=0.6):
    """
    Compare input image against stored embeddings.
    Returns (best_match_name, similarity_score).
    """
    emb = get_face_embedding(image_path)
    if emb is None:
        return None, 0.0

    # Compute cosine similarity between this and all stored embeddings
    sims = cosine_similarity([emb], stored_embeddings)[0]
    best_idx = np.argmax(sims)
    best_score = sims[best_idx]
    best_name = stored_names[best_idx]

    if best_score >= threshold:
        print(f"Match: {best_name} (confidence {best_score:.3f})")
        return best_name, best_score
    else:
        print(f"Unknown face (max similarity {best_score:.3f})")
        return "Unknown", best_score


In [ ]:

MODEL_PATH = "inception_resnet_v1_vggface2.pth"
torch.save(resnet.state_dict(), MODEL_PATH)
print("✅ Model saved at:", MODEL_PATH)

✅ Model saved at: inception_resnet_v1_vggface2.pth


In [ ]:
from google.colab import files
files.download("inception_resnet_v1_vggface2.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>